Importing Libraries

In [9]:
import sys
import os
import numpy as np
import renders as rs
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import renders as rs
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.cluster import KMeans
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model

/Users/senzari/Machine_Learning/stats/src


In [10]:
import pandas as pd 
# Reading in a CSV adds the first index column
training_data = pd.read_csv('round25.csv')
training_data = training_data.drop(training_data.columns[[0]], axis=1)

Find all matches

In [ ]:
# predictor_model = form_model.build_model(training_data)

# joblib.dump(predictor_model, 'pn.pkl')
predictor_model = joblib.load('pn.pkl')

In [ ]:
from stats import predict_matches
predictions, upcoming_matches = predict_matches.predictions(predictor_model)

In [ ]:
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones abovev
upcoming_matches = upcoming_matches[columns]
# Add predictions to the end of that DF
upcoming_matches = upcoming_matches.join(pd.DataFrame(df))
upcoming_matches = upcoming_matches[(upcoming_matches['scheduled'] < '2016-08-26')]
upcoming_matches.to_csv('predictions_08_24_2016.csv')

In [16]:
"""X, y = form_model.get_split(training_data)
pca = PCA(n_components=6)
pca.fit(X)
# Generate PCA results plot
pca_results = rs.pca_results(X, pca)"""

'X, y = form_model.get_split(training_data)\npca = PCA(n_components=6)\npca.fit(X)\n# Generate PCA results plot\npca_results = rs.pca_results(X, pca)'

In [3]:
# sil_scores, predictor_model = form_model.build_model(training_data, 'gmm')
# joblib.dump(predictor_model, 'gmm.pkl')
# Can we tell by the centers of the data the results of the game
# true_centers = pd.DataFrame(np.round(centers), columns = reduced_data.keys())

(3, 'clusters:', 0.1182738784917476)


['gmm.pkl', 'gmm.pkl_01.npy', 'gmm.pkl_02.npy', 'gmm.pkl_03.npy']

In [7]:
predictor_model = joblib.load('gmm.pkl')
from stats import predict_matches
predictions, upcoming_matches = predict_matches.predictions(predictor_model)
print(predictions)
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones abovev
upcoming_matches = upcoming_matches[columns]
# Add predictions to the end of that DF
upcoming_matches = upcoming_matches.join(pd.DataFrame(predictions))
upcoming_matches = upcoming_matches[(upcoming_matches['scheduled'] < '2016-08-26')]
upcoming_matches.to_csv('predictions_gmm_08_24_2016.csv')

[1 1 1 0 0 2 1 2 0 1 0 0 1 2 2 0 0 0 0 0 2 2 0 0 1 1 1 1 1 2 1 1]


In [ ]:
# Display the results of the clustering from implementation
# rs.cluster_results(reduced_data, preds, centers)

In [11]:
X, y = form_model.get_split(training_data)
kmeans = KMeans(init='k-means++', n_clusters=3, n_init=10)
predictor_model = kmeans.fit(X)
joblib.dump(predictor_model, 'kmeans.pkl')

['kmeans.pkl', 'kmeans.pkl_01.npy', 'kmeans.pkl_02.npy']

In [12]:
predictor_model = joblib.load('kmeans.pkl')
from stats import predict_matches
predictions, upcoming_matches = predict_matches.predictions(predictor_model)
print(predictions)
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones abovev
upcoming_matches = upcoming_matches[columns]
# Add predictions to the end of that DF
upcoming_matches = upcoming_matches.join(pd.DataFrame(predictions))
upcoming_matches = upcoming_matches[(upcoming_matches['scheduled'] < '2016-08-26')]
upcoming_matches.to_csv('predictions_kmeans_08_24_2016.csv')

[0 0 0 0 0 2 0 0 1 0 1 1 0 1 0 0 2 0 0 2 0 0 1 2 0 0 0 0 0 0 0 0]
